In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append("..")

In [3]:
from optimus import Optimus

In [4]:
# Create optimus
op = Optimus()


                             ____        __  _                     
                            / __ \____  / /_(_)___ ___  __  _______
                           / / / / __ \/ __/ / __ `__ \/ / / / ___/
                          / /_/ / /_/ / /_/ / / / / / / /_/ (__  ) 
                          \____/ .___/\__/_/_/ /_/ /_/\__,_/____/  
                              /_/                                  
                              
Transform and Roll out...
Just checking that all necessary environments vars are present...
-----
PYSPARK_PYTHON=python
SPARK_HOME=C:\opt\spark\spark-2.3.1-bin-hadoop2.7
JAVA_HOME=C:\java8
Pyarrow Installed
-----
Starting or getting SparkSession and SparkContext...
Setting checkpoint folder local. If you are in a cluster initialize Optimus with master='your_ip' as param
Deleting previous folder if exists...
Creating the checkpoint directory...
Optimus successfully imported. Have fun :).


In [5]:
from pyspark.sql.types import StructType, StructField, StringType, BooleanType, IntegerType, ArrayType

df =op.load.url("https://raw.githubusercontent.com/ironmussa/Optimus/master/examples/foo.csv")

Downloaded 967 bytes
Creating DataFrame for foo.csv. Please wait...
Loading file using SparkSession
Successfully created DataFrame for 'foo.csv'


### Quantile Statistics

In [6]:
print(df.cols.min("billingId"))
print(df.cols.percentile(['billingId', 'price'], [0.05, 0.25, 0.5, 0.75, 0.95]))
print(df.cols.max("billingId"))
print(df.cols.median(["billingId","price"]))
print(df.cols.range(["billingId","price"]))
print(df.cols.std(["billingId","price"]))

min function took 1538.694 ms
111
drop_na function took 12.965 ms
cast function took 11.915 ms
drop_na function took 5.984 ms
cast function took 7.978 ms
percentile function took 270.276 ms
{'billingId': {0.05: 111.0, 0.25: 323.0, 0.5: 553.0, 0.75: 812.0, 0.95: 992.0}, 'price': {0.05: 1.0, 0.25: 3.0, 0.5: 8.0, 0.75: 8.0, 0.95: 10.0}}
max function took 1130.199 ms
992
drop_na function took 6.981 ms
cast function took 7.944 ms
drop_na function took 4.987 ms
cast function took 7.948 ms
median function took 179.520 ms
{'billingId': 553.0, 'price': 8.0}
max function took 1097.800 ms
min function took 1109.014 ms
max function took 1124.247 ms
min function took 1116.700 ms
range function took 4448.723 ms
{'billingId': {'min': 111, 'max': 992}, 'price': {'min': 1, 'max': 10}}
std function took 1191.743 ms
{'stddev_billingId': 280.1973510859008, 'stddev_price': 2.9528457876452054}


In [7]:
print(df.cols.min("*"))

min function took 1229.156 ms
{'min_id': 1, 'min_firstName': '(((   Heinrich )))))', 'min_lastName': '             GALiLEI', 'min_billingId': 111, 'min_product': '110790', 'min_price': 1, 'min_birth': '1899/01/01', 'min_dummyCol': '#'}


### Descriptive Statistics

In [8]:
print(df.cols.kurt("billingId"))
print(df.cols.mean("billingId"))
print(df.cols.skewness("billingId"))
print(df.cols.sum("billingId"))
print(df.cols.variance("billingId"))
print(df.cols.mad("billingId"))

kurt function took 1198.001 ms
-1.0411548120618528
mean function took 1171.006 ms
556.0
skewness function took 1150.990 ms
-0.2137018086949909
sum function took 1129.183 ms
10564
variance function took 1154.769 ms
78510.55555555555
drop_na function took 7.979 ms
cast function took 6.979 ms
median function took 82.747 ms
drop_na function took 2.992 ms
cast function took 4.932 ms
median function took 98.736 ms
mad function took 243.318 ms
230.0


In [9]:
df.cols.mad("price")

drop_na function took 5.954 ms
cast function took 7.979 ms
median function took 80.753 ms
drop_na function took 3.991 ms
cast function took 10.009 ms
median function took 101.727 ms
mad function took 233.340 ms


2.0

In [10]:
df.cols.mad("price", more= True)

drop_na function took 6.022 ms
cast function took 7.980 ms
median function took 93.750 ms
drop_na function took 3.990 ms
cast function took 5.985 ms
median function took 76.794 ms
mad function took 227.392 ms


{'mad': 2.0, 'median': 8.0}

In [11]:
print(df.cols.percentile(['price'], [0.05, 0.25, 0.5, 0.75, 0.95]))

drop_na function took 4.951 ms
cast function took 10.911 ms
percentile function took 79.752 ms
{0.05: 1.0, 0.25: 3.0, 0.5: 8.0, 0.75: 8.0, 0.95: 10.0}


In [12]:
print(df.cols.mode(["price","billingId"]))

mode function took 11922.792 ms
[{'price': 8}, {'billingId': None}]


### String Operations

In [13]:
df.show() 

+---+--------------------+--------------------+---------+----------+-----+----------+--------+
| id|           firstName|            lastName|billingId|   product|price|     birth|dummyCol|
+---+--------------------+--------------------+---------+----------+-----+----------+--------+
|  1|                Luis|         Alvarez$$%!|      123|      Cake|   10|1980/07/07|   never|
|  2|               André|              Ampère|      423|      piza|    8|1950/07/08|   gonna|
|  3|               NiELS|          Böhr//((%%|      551|     pizza|    8|1990/07/09|    give|
|  4|                PAUL|              dirac$|      521|     pizza|    8|1954/07/10|     you|
|  5|              Albert|            Einstein|      634|     pizza|    8|1990/07/11|      up|
|  6|             Galileo|             GALiLEI|      672|     arepa|    5|1930/08/12|   never|
|  7|                CaRL|            Ga%%%uss|      323|      taco|    3|1970/07/13|   gonna|
|  8|               David|          H$$$ilbert|   

In [14]:
df\
    .cols.trim("lastName")\
    .cols.lower("lastName")\
    .cols.upper(["product", "firstName"])\
    .cols.reverse("firstName")\
    .show()

Using 'Column Expression' to process column 'lastName'
trim function took 5.984 ms
Using 'Column Expression' to process column 'lastName'
lower function took 8.975 ms
Using 'Column Expression' to process column 'product'
Using 'Column Expression' to process column 'firstName'
upper function took 12.965 ms
Using 'Column Expression' to process column 'firstName'
reverse function took 5.980 ms
+---+--------------------+-----------+---------+----------+-----+----------+--------+
| id|           firstName|   lastName|billingId|   product|price|     birth|dummyCol|
+---+--------------------+-----------+---------+----------+-----+----------+--------+
|  1|                SIUL|alvarez$$%!|      123|      CAKE|   10|1980/07/07|   never|
|  2|               ÉRDNA|     ampère|      423|      PIZA|    8|1950/07/08|   gonna|
|  3|               SLEIN| böhr//((%%|      551|     PIZZA|    8|1990/07/09|    give|
|  4|                LUAP|     dirac$|      521|     PIZZA|    8|1954/07/10|     you|
|  5

In [15]:
df.cols.iqr("price")

drop_na function took 5.986 ms
cast function took 9.972 ms
percentile function took 83.776 ms
iqr function took 84.774 ms


5.0

In [16]:
df.cols.iqr("price", more= True)

drop_na function took 5.984 ms
cast function took 10.972 ms
percentile function took 91.757 ms
iqr function took 92.753 ms


{'iqr': 5.0, 'q1': 3.0, 'q3': 8.0}

In [17]:
df.cols.z_score("price").show()

mean function took 1091.136 ms
std function took 1166.092 ms
z_score function took 2313.077 ms
+---+--------------------+--------------------+---------+----------+-----+----------+--------+------------------+
| id|           firstName|            lastName|billingId|   product|price|     birth|dummyCol|       z_col_price|
+---+--------------------+--------------------+---------+----------+-----+----------+--------+------------------+
|  1|                Luis|         Alvarez$$%!|      123|      Cake|   10|1980/07/07|   never|1.3368014129178498|
|  2|               André|              Ampère|      423|      piza|    8|1950/07/08|   gonna|0.6594886970394727|
|  3|               NiELS|          Böhr//((%%|      551|     pizza|    8|1990/07/09|    give|0.6594886970394727|
|  4|                PAUL|              dirac$|      521|     pizza|    8|1954/07/10|     you|0.6594886970394727|
|  5|              Albert|            Einstein|      634|     pizza|    8|1990/07/11|      up|0.65948869703